## Script responsável por realizar a lematização

- Este código pressupõe que a existência do arquivo **.\saida\03_2_DouSecao0<No Seção>_portarias_mgi_tratado_NER.parquet** 
- Caso não exista, ele pode ser gerado por meio do notebook **03_2_Tratamento_RemocaoNomesProprios.ipynb**
- Processamento deste script:
  - Realiza a lematização, **coluna TextoTratadoSemNomes**, mantendo os números
  - Armazena na **coluna TextoTratadoSemNomesLemmatized**
  - Cria a **coluna TextoTratadoSemNomesLemmatized_TotalPalavras** com o total do palavras do texto tratado
  
  
---


## Bibliotecas

In [1]:
# Biblioteca para manipulação e análise de dados
import pandas as pd

# Bibliotecas para processamento de texto
import re  # Para operações com expressões regulares
import spacy  # Para tarefas avançadas de processamento de linguagem natural

# Biblioteca para medir o tempo de execução
import time

# Bibliotecas NLTK para processamento de linguagem natural
import nltk
from nltk.corpus import stopwords  # Para uso de listas de palavras de parada

# Certifique-se de baixar as stopwords, caso ainda não tenha feito isso
# nltk.download('stopwords')


In [2]:
douItem = 2
secao = f'Secao0{douItem}'

# Lê o arquivo Parquet
df_portarias_mgi = pd.read_parquet(f'./saida/03_2_DOU{secao}_portarias_mgi_tratado_NER.parquet', engine='pyarrow')  # ou engine='fastparquet'

df_portarias_mgi.shape


(19258, 36)

In [3]:
df_portarias_mgi.head()

,id,name,idOficio,pubName,artType,pubDate,artClass,artCategory,artSize,artNotes,...,zip_name,Ano,Mes,Texto,Total_palavras,TextoTratado,TextoTratado_TotalPalavras,Texto_sem_nomes,Nomes,TextoTratadoSemNomes
0,30266027,PORTARIA DE PESSOAL 529,9368436,DO2,Portaria,2023-01-25,00028:00006:00000:00000:00000:00000:00000:0000...,Ministério da Gestão e da Inovação em Serviços...,12,,...,S02012023.zip,2023,1,"PORTARIA DE PESSOAL SGP/SEDGG/ME Nº 529, DE 17...",358,"sgp/sedgg/me nº 529, 17 janeiro 2023 secretari...",231,None,"sergio eduardo arbulu, anderson pereira silvy,...","sgp / sedgg / me nº 529 , 17 janeiro 2023 secr..."
1,30264410,ATO PORTARIA 655,9368530,DO2,Portaria,2023-01-25,00028:00006:00000:00000:00000:00000:00000:0000...,Ministério da Gestão e da Inovação em Serviços...,12,,...,S02012023.zip,2023,1,"PORTARIA SGP/SEDGG/ME Nº 655, DE 23 DE JANEIRO...",245,"portaria sgp/sedgg/me nº 655, 23 janeiro 2023 ...",158,None,"eliane araujo franklin, eduardo arbulu","portaria sgp / sedgg / me nº 655 , 23 janeiro ..."
2,30265009,ATO PORTARIA DE PESSOAL 461,9368536,DO2,Portaria,2023-01-25,00028:00006:00000:00000:00000:00000:00000:0000...,Ministério da Gestão e da Inovação em Serviços...,12,,...,S02012023.zip,2023,1,"PORTARIA DE PESSOAL SGP/SEDGG/ME Nº 461, DE 16...",251,"sgp/sedgg/me nº 461, 16 janeiro 2023 secretari...",160,None,eduardo arbulu,"sgp / sedgg / me nº 461 , 16 janeiro 2023 secr..."
3,30265678,ATO PORTARIA DE PESSOAL SGP 606,9368541,DO2,Portaria,2023-01-25,00028:00006:00000:00000:00000:00000:00000:0000...,Ministério da Gestão e da Inovação em Serviços...,12,,...,S02012023.zip,2023,1,"PORTARIA DE PESSOAL SGP/SEDGG/ME Nº 606, DE 19...",251,"sgp/sedgg/me nº 606, 19 janeiro 2023 secretari...",158,None,eduardo arbulu,"sgp / sedgg / me nº 606 , 19 janeiro 2023 secr..."
4,30266030,ATO PORTARIA DE PESSOAL SGP 517,9368573,DO2,Portaria,2023-01-25,00028:00006:00000:00000:00000:00000:00000:0000...,Ministério da Gestão e da Inovação em Serviços...,12,,...,S02012023.zip,2023,1,"PORTARIA DE PESSOAL SGP/SEDGG/ME Nº 517, DE 17...",243,"sgp/sedgg/me nº 517, 17 janeiro 2023 secretari...",158,None,"roberto jorge ferreira, termino exercicio, edu...","sgp / sedgg / me nº 517 , 17 janeiro 2023 secr..."


In [4]:
# Criar a nova coluna com o total de palavras
df_portarias_mgi['TextoTratadoSemNomes_TotalPalavras'] = df_portarias_mgi['TextoTratadoSemNomes'].apply(
    lambda texto: len(str(texto).split())
)

In [5]:
# Carregar o modelo de português do spaCy
nlp = spacy.load("pt_core_news_sm")

# Função de pré-processamento e lematização com spaCy
def preprocess_with_spacy(text):
    """
    Processa o texto para lematização usando spaCy.
    
    Args:
        text (str): O texto a ser processado.
        
    Returns:
        str: O texto lematizado com números preservados.
    """
    # Processar o texto com spaCy
    doc = nlp(text.lower())
    
    # Extrair palavras lematizadas, removendo stopwords, mas preservando números
    lemmatized_text = " ".join(
        token.lemma_ for token in doc if not token.is_stop and (token.is_alpha or token.is_digit)
    )
    
    return lemmatized_text

# Processar o DataFrame em blocos
def process_in_blocks(df, block_size=200):
    """
    Processa o DataFrame em blocos para aplicar a lematização.
    
    Args:
        df (DataFrame): DataFrame contendo os textos a serem processados.
        block_size (int): Quantidade de registros por bloco.
        
    Returns:
        DataFrame: DataFrame com os textos lematizados.
    """
    processed_blocks = []
    for i in range(0, len(df), block_size):
        print(f"Processando registros de índice {i} até {min(i + block_size, len(df)) - 1}...")
        
        # Selecionar o bloco atual
        block = df.iloc[i:i + block_size].copy()
        
        # Processar o bloco e adicionar coluna lematizada
        block['TextoTratadoSemNomesLemmatized'] = block['TextoTratadoSemNomes'].apply(preprocess_with_spacy)
        
        # Adicionar o bloco processado à lista
        processed_blocks.append(block)
    
    return pd.concat(processed_blocks, ignore_index=True)

# Medir o tempo total do processamento
start_time = time.time()

print("Início do processamento em blocos...")
df_portarias_mgi = process_in_blocks(df_portarias_mgi, block_size=200)

end_time = time.time()

# Exibir o tempo total de processamento
total_time = end_time - start_time
print(f"\nTempo total de processamento: {total_time:.2f} segundos.")


Início do processamento em blocos...
Processando registros de índice 0 até 199...
Processando registros de índice 200 até 399...
Processando registros de índice 400 até 599...
Processando registros de índice 600 até 799...
Processando registros de índice 800 até 999...
Processando registros de índice 1000 até 1199...
Processando registros de índice 1200 até 1399...
Processando registros de índice 1400 até 1599...
Processando registros de índice 1600 até 1799...
Processando registros de índice 1800 até 1999...
Processando registros de índice 2000 até 2199...
Processando registros de índice 2200 até 2399...
Processando registros de índice 2400 até 2599...
Processando registros de índice 2600 até 2799...
Processando registros de índice 2800 até 2999...
Processando registros de índice 3000 até 3199...
Processando registros de índice 3200 até 3399...
Processando registros de índice 3400 até 3599...
Processando registros de índice 3600 até 3799...
Processando registros de índice 3800 até 399

In [6]:
# Criar a nova coluna com o total de palavras
df_portarias_mgi['TextoTratadoSemNomesLemmatized_TotalPalavras'] = df_portarias_mgi['TextoTratadoSemNomesLemmatized'].apply(
    lambda texto: len(str(texto).split())
)

In [7]:
df_portarias_mgi.shape

(19258, 39)

In [8]:
df_portarias_mgi.head()

,id,name,idOficio,pubName,artType,pubDate,artClass,artCategory,artSize,artNotes,...,Texto,Total_palavras,TextoTratado,TextoTratado_TotalPalavras,Texto_sem_nomes,Nomes,TextoTratadoSemNomes,TextoTratadoSemNomes_TotalPalavras,TextoTratadoSemNomesLemmatized,TextoTratadoSemNomesLemmatized_TotalPalavras
0,30266027,PORTARIA DE PESSOAL 529,9368436,DO2,Portaria,2023-01-25,00028:00006:00000:00000:00000:00000:00000:0000...,Ministério da Gestão e da Inovação em Serviços...,12,,...,"PORTARIA DE PESSOAL SGP/SEDGG/ME Nº 529, DE 17...",358,"sgp/sedgg/me nº 529, 17 janeiro 2023 secretari...",231,None,"sergio eduardo arbulu, anderson pereira silvy,...","sgp / sedgg / me nº 529 , 17 janeiro 2023 secr...",268,sgp Sedgg nº 529 17 janeiro 2023 secretario ge...,182
1,30264410,ATO PORTARIA 655,9368530,DO2,Portaria,2023-01-25,00028:00006:00000:00000:00000:00000:00000:0000...,Ministério da Gestão e da Inovação em Serviços...,12,,...,"PORTARIA SGP/SEDGG/ME Nº 655, DE 23 DE JANEIRO...",245,"portaria sgp/sedgg/me nº 655, 23 janeiro 2023 ...",158,None,"eliane araujo franklin, eduardo arbulu","portaria sgp / sedgg / me nº 655 , 23 janeiro ...",189,portar sgp Sedgg nº 655 23 janeiro 2023 secret...,127
2,30265009,ATO PORTARIA DE PESSOAL 461,9368536,DO2,Portaria,2023-01-25,00028:00006:00000:00000:00000:00000:00000:0000...,Ministério da Gestão e da Inovação em Serviços...,12,,...,"PORTARIA DE PESSOAL SGP/SEDGG/ME Nº 461, DE 16...",251,"sgp/sedgg/me nº 461, 16 janeiro 2023 secretari...",160,None,eduardo arbulu,"sgp / sedgg / me nº 461 , 16 janeiro 2023 secr...",197,sgp Sedgg nº 461 16 janeiro 2023 secretario ge...,132
3,30265678,ATO PORTARIA DE PESSOAL SGP 606,9368541,DO2,Portaria,2023-01-25,00028:00006:00000:00000:00000:00000:00000:0000...,Ministério da Gestão e da Inovação em Serviços...,12,,...,"PORTARIA DE PESSOAL SGP/SEDGG/ME Nº 606, DE 19...",251,"sgp/sedgg/me nº 606, 19 janeiro 2023 secretari...",158,None,eduardo arbulu,"sgp / sedgg / me nº 606 , 19 janeiro 2023 secr...",187,sgp Sedgg nº 606 19 janeiro 2023 secretario ge...,134
4,30266030,ATO PORTARIA DE PESSOAL SGP 517,9368573,DO2,Portaria,2023-01-25,00028:00006:00000:00000:00000:00000:00000:0000...,Ministério da Gestão e da Inovação em Serviços...,12,,...,"PORTARIA DE PESSOAL SGP/SEDGG/ME Nº 517, DE 17...",243,"sgp/sedgg/me nº 517, 17 janeiro 2023 secretari...",158,None,"roberto jorge ferreira, termino exercicio, edu...","sgp / sedgg / me nº 517 , 17 janeiro 2023 secr...",177,sgp Sedgg nº 517 17 janeiro 2023 secretario ge...,122


In [9]:
# Salva saída parcial o arquivo completo com novas colunas
df_portarias_mgi.to_parquet(f'./saida/03_3_DOU{secao}_portarias_mgi_tratado_NER_lematizado.parquet', engine='pyarrow', index=False)

In [10]:
# output_path = "df_portarias_mgi_first_1000.xlsx"
# df_portarias_mgi.head(1000).to_excel(output_path, index=False)